# Vector fields and developmental trajectories of tissue development in three-dimensional space

In [2]:
import os, warnings
from pathlib import Path

import numpy as np
import dynamo as dyn
import spateo as st
import scanpy as sc
warnings.filterwarnings('ignore')

## Load the data

In [3]:
sample_id = "E7_8h"
os.chdir(f"/media/pc001/Yao/Projects/Project_spateo/mouse_heart/figure6")
cpo = [(41, 1209, 57), (13, 8, -3), (0, 0, 1)]

out_h5ad_path = f"droso_{sample_id}_CNS_morpho/h5ad"
out_image_path = f"droso_{sample_id}_CNS_morpho/image/CNS_morphofield"
Path(out_h5ad_path).mkdir(parents=True, exist_ok=True)
Path(out_image_path).mkdir(parents=True, exist_ok=True)

In [4]:
germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E7_8h_germ_layer_new/h5ad/{sample_id}_germband_v3.h5ad")
tissue_adata = germ_adata[germ_adata.obs["anno_tissue_new"] == "CNS", :]
tissue_adata.X = tissue_adata.layers["X_counts"].copy()
tissue_adata = tissue_adata[tissue_adata.X.sum(axis=1)!=0, tissue_adata.X.sum(axis=0)!=0]
tissue_adata

In [5]:
tissue_pc, _ = st.tdr.construct_pc(
    adata=tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)

## Check mapping result

In [26]:
stage2_germ_adata = st.read_h5ad(f"/media/pc001/Yao/Projects/Project_drosophila/Data_v4/migration-hotspot/drosophila_E8_9h_germ_layer/h5ad/E8_9h_germ_layer_v3.h5ad")
stage2_tissue_adata = stage2_germ_adata[stage2_germ_adata.obs["anno_tissue_new"] == "CNS", :]
stage2_tissue_adata.X = stage2_tissue_adata.layers["X_counts"].copy()
stage2_tissue_pc, _ = st.tdr.construct_pc(
    adata=stage2_tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
models_distance = np.asarray([0, 0, -260])
stage2_tissue_pc.points = stage2_tissue_pc.points + models_distance
stage2_tissue_adata

In [27]:
model_lines, _ = st.tdr.construct_align_lines(
            model1_points=tissue_adata.obsm["3d_align_spatial"].copy(),
            model2_points=tissue_adata.obsm["X_cells_mapping"] + models_distance,
            key_added="mapping",
            label="lines",
            color="gainsboro",
            alpha=1.0,
        )

mapping_cpo = [(18, 1370, -141),(-1.8, 5.8, -156.8), (0, 0, 1)]
st.pl.three_d_plot(
        model=st.tdr.collect_models([model_lines, st.tdr.merge_models([tissue_pc, stage2_tissue_pc])]),
        key=["mapping", "tissue"],
        jupyter="static",
        cpo=mapping_cpo,
        background="white",
        window_size=(2560, 2048),
        opacity=[0.1, 1],
        model_style=["wireframe", "points"],
        model_size=[2, 16],
        filename=os.path.join(out_image_path, f"E7-8h-E8-9h_CNS_mapping_z_model_260.pdf"),
    )

## Calculate the developmental Vectorfield

In [6]:
st.tdr.morphofield_sparsevfc(
    adata=tissue_adata,
    spatial_key="3d_align_spatial",
    V_key="V_cells_mapping",
    key_added="VecFld_morpho",
    NX=np.asarray(tissue_adata.obsm['3d_align_spatial']),
    inplace=True,
)

tissue_pc.point_data["vectors"] = tissue_adata.uns["VecFld_morpho"]["V"]

In [24]:
for i, k in zip([0, 2], ["V_x", "V_z"]):
    vector_arrows1,_ = st.tdr.construct_field(
        model=tissue_pc,
        vf_key="vectors",
        arrows_scale_key="vectors",
        n_sampling=None,
        factor=3000,
        key_added=k,
        label=tissue_pc.point_data["vectors"][:, i].flatten(),
        color= "rainbow",
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, vector_arrows1]),
        key=["tissue", k],
        colormap=["gainsboro", "Spectral"],
        opacity=[0.2, 1],
        model_style=["points", "surface"],
        model_size=[16, 5],
        jupyter="static",
        background="white",
        window_size=(2560, 2048),
        cpo=cpo,
        filename=os.path.join(out_image_path, f"CNS_morphofield_vector_arrows_pc_model_{k}.pdf")
    )

## Predict the cell developmentalal trajectory

In [6]:
st.tdr.morphopath(
    adata=tissue_adata,
    # layer="log1p_X",
    vf_key="VecFld_morpho",
    key_added="fate_morpho",
    t_end=5000,
    interpolation_num=50,
    cores=20
)

In [23]:
trajectory_model, _ = st.tdr.construct_trajectory(
    adata=tissue_adata,
    fate_key="fate_morpho",
    n_sampling=1000,
    sampling_method="trn",
    key_added="obs_index",
    label=np.asarray(tissue_adata.obs.index), # stage1_tissue_adata.uns["VecFld_morpho"]["V"][:, 2].flatten(),
)
for i, k in zip([0, 2], ["V_x", "V_z"]):
    tissue_adata.obs[k] = tissue_adata.uns["VecFld_morpho"]["V"][:, i].flatten()
    st.tdr.add_model_labels(
        model=trajectory_model,
        key_added=k,
        labels=np.asarray(tissue_adata[np.asarray(trajectory_model.point_data["obs_index"])].obs[k]),
        colormap="Spectral",
        where="point_data",
        inplace=True,
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, trajectory_model]),
        key=["tissue", k],
        opacity=[0.2, .5],
        model_style=["points", "wireframe"],
        model_size=[16, 5],
        colormap=["gainsboro", "Spectral"],
        jupyter="static",
        background="white",
        window_size=(2560, 2048),
        cpo=cpo,
        filename=os.path.join(out_image_path, f"CNS_morphofield_trajectory_pc_model_{k}.pdf")
    )

## Animating cell fate prediction

In [11]:
cells_models, _ = st.tdr.construct_genesis(
    adata=tissue_adata,
    fate_key="fate_morpho",
    n_steps = 100,
    logspace=True,
    t_end=4000,
    label=[tissue_adata.uns["VecFld_morpho"]["V"][:, 0]] * 100,
    color="Spectral"
)
st.pl.three_d_animate(
    models=cells_models,
    stable_model=trajectory_model,
    key="genesis",
    stable_kwargs=dict(
        key="V_x",
        model_style="wireframe",
        model_size=5,
        opacity=0.5,
        colormap="Spectral",
        show_legend=False,
    ),
    filename=os.path.join(out_image_path, f"CNS_morphofield_trajectory_model.mp4"),
    colormap="Spectral",
    model_style="points",
    model_size=16,
    jupyter="static",
    background="white",
    window_size=(2560, 2048),
    cpo=cpo,
    framerate=10)

In [12]:
cells_linear_models, _ = st.tdr.construct_genesis_X(
    stages_X=[tissue_adata.obsm["3d_align_spatial"], tissue_adata.obsm["X_cells_mapping"]],
    n_spacing=100,
    key_added="genesis",
    color="Spectral", 
    label=[tissue_adata.uns["VecFld_morpho"]["V"][:, 0]] * (100 + 1),
)
st.pl.three_d_animate(
    models=cells_linear_models,
    stable_model=trajectory_model,
    key="genesis",
    stable_kwargs=dict(
        key="V_x",
        model_style="wireframe",
        model_size=5,
        opacity=0.5,
        colormap="Spectral",
        show_legend=False,
    ),
    filename=os.path.join(out_image_path, f"CNS_morphofield_trajectory_model_linear.mp4"),
    model_style="points",
    model_size=16,
    colormap="Spectral",
    jupyter="static",
    background="white",
    window_size=(2560, 2048),
    cpo=cpo,
    framerate=10)

## Segments of CNS

In [21]:
germ_pc, _ = st.tdr.construct_pc(
    adata=germ_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="segments",
    key_added="segments",
    colormap="Spectral",
)
st.pl.three_d_plot(
        model=st.tdr.collect_models([tissue_pc, germ_pc]),
        key=["tissue", "segments"],
        opacity=[0.8, .1],
        model_style=["points", "points"],
        model_size=[16, 20],
        colormap=[None, "Spectral"],
        jupyter="static",
        background="white",
        window_size=(2560, 2048),
        cpo=cpo,
        filename=os.path.join(out_image_path, f"CNS_segments_background_pc_model.pdf")
    )

In [20]:
pc_index = tissue_pc.point_data["obs_index"].tolist()
st.tdr.add_model_labels(model=tissue_pc, labels=tissue_adata.obs.loc[pc_index, "segments"], key_added="segments", where="point_data", colormap="Spectral", inplace=True)
st.pl.three_d_plot(
        model=tissue_pc,
        key="segments",
        opacity=1,
        model_style="points",
        model_size=16,
        colormap="Spectral",
        jupyter="static",
        background="white",
        window_size=(2560, 2048),
        cpo=cpo,
        filename=os.path.join(out_image_path, f"CNS_segments_pc_model.pdf")
    )

In [7]:
morphometrics_keys = ["morpho_acceleration"]
st.tdr.morphofield_velocity(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_velocity", inplace=True)
st.tdr.morphofield_acceleration(adata=tissue_adata, vf_key="VecFld_morpho", key_added="morpho_acceleration", inplace=True)
tissue_adata

In [50]:
for mk in morphometrics_keys:
    #tissue_adata.obs[mk] = tissue_adata.obs[mk].map(lambda x: 7e-7 if x >= 7e-7 else x)
    #tissue_adata.obs[mk] = tissue_adata.obs[mk].map(lambda x: 1e-7 if x <= 1e-7 else x)
    st.tdr.add_model_labels(
        model = trajectory_model,
        key_added = mk,
        labels = np.asarray(tissue_adata[np.asarray(trajectory_model.point_data["obs_index"])].obs[mk]),
        colormap = "afmhot_r",
        where = "point_data",
        inplace = True,
        )
    st.tdr.add_model_labels(
        model = tissue_pc,
        key_added = mk,
        labels = np.asarray(tissue_adata[np.asarray(tissue_pc.point_data["obs_index"])].obs[mk]),
        colormap = "afmhot_r",
        where = "point_data",
        inplace = True,
    )
    
    st.pl.three_d_plot(
        model = st.tdr.collect_models([tissue_pc, trajectory_model]),
        key = [mk, mk],
        model_style = ["points", "wireframe"],
        model_size = [16, 5],
        opacity = [0.2, 0.5],
        colormap = ["afmhot_r", "afmhot_r"],
        show_legend = True,
        jupyter = "static",
        background = "white",
        cpo = cpo,
        window_size = (2560, 2048),
        text = mk,
        filename = os.path.join(out_image_path, f"CNS_morphometrics_{mk}_trajectory.pdf")
    )

In [10]:
import matplotlib as mpl
lscmap = mpl.cm.get_cmap("Spectral") # gist_rainbow, rainbow, hsv, tab20
segments = ['abdomen', 'head', "thorax"]
segments_hex_list = [mpl.colors.to_hex(lscmap(i)) for i in np.linspace(0, 1, len(segments))]
segments_colors = {i: j for i, j in zip(segments, segments_hex_list)}

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
acc_data = pd.DataFrame(tissue_adata.obs[["segments", "morpho_acceleration"]])

sns.set_theme(context="paper", style="ticks")
f, ax = plt.subplots(figsize=(1.5, 1.5))
sns.boxenplot(acc_data, x="segments", y="morpho_acceleration", palette=segments_colors, order=['head', "thorax", 'abdomen'], 
              linewidth=.5, line_kws=dict(linewidth=2, color="k"), flier_kws=dict(facecolor=".5", linewidth=.1), width=.5)
# sns.violinplot(data, x="heart_regions", y=key, palette=regions_colors, order=regions, ax=ax)
ax.tick_params(axis='x', labelrotation=90)
plt.savefig(os.path.join(out_image_path, f"CNS_morphometrics_morpho_acceleration_segments_boxplot.pdf"), dpi=300, format="pdf", bbox_inches="tight")
plt.show()

## Tail of CNS

In [51]:
tail_tissue_adata = tissue_adata.copy()
tail_tissue_adata = tail_tissue_adata[tail_tissue_adata.obs["segments"].isin(["abdomen"]), :].copy()
tail_tissue_adata

In [52]:
st.tdr.morphofield_sparsevfc(
    adata=tail_tissue_adata,
    spatial_key="3d_align_spatial",
    V_key="V_cells_mapping",
    key_added="VecFld_morpho",
    NX=np.asarray(tail_tissue_adata.obsm['3d_align_spatial']),
    inplace=True,
)

tail_tissue_pc, _ = st.tdr.construct_pc(
    adata=tail_tissue_adata.copy(),
    spatial_key="3d_align_spatial",
    groupby="anno_tissue_new",
    key_added="tissue",
    colormap={"amnioserosa": "#bdcf32", "CNS": "#ea5545", "hindgut": "#27aeef", "midgut": "#87bc45", "muscle": "#f46a9b", "salivary gland": "#ef9b20",},
)
tail_tissue_pc.point_data["vectors"] = tail_tissue_adata.uns["VecFld_morpho"]["V"]

In [53]:
st.tdr.morphofield_velocity(adata=tail_tissue_adata, vf_key="VecFld_morpho", key_added="morpho_velocity",)
st.tdr.morphofield_acceleration(adata=tail_tissue_adata, vf_key="VecFld_morpho", key_added="morpho_acceleration",)
st.tdr.morphofield_curvature(adata=tail_tissue_adata, vf_key="VecFld_morpho", key_added="morpho_curvature")
st.tdr.morphofield_curl(adata=tail_tissue_adata, vf_key="VecFld_morpho", key_added="morpho_curl",)
st.tdr.morphofield_torsion(adata=tail_tissue_adata, vf_key="VecFld_morpho", key_added="morpho_torsion",)
st.tdr.morphofield_divergence(adata=tail_tissue_adata, vf_key="VecFld_morpho", key_added="morpho_divergence",)
tail_tissue_adata

In [56]:
morphometrics_keys = ["morpho_acceleration", "morpho_curvature", "morpho_curl", "morpho_torsion", "morpho_divergence"]
for mk in morphometrics_keys:
    vector_arrows,_ = st.tdr.construct_field(
        model=tail_tissue_pc,
        vf_key="vectors",
        arrows_scale_key="vectors",
        n_sampling=1000,
        factor=10000,
        key_added=mk,
        label=np.asarray(tail_tissue_adata[np.asarray(tail_tissue_pc.point_data["obs_index"])].obs[mk]),
        color="afmhot_r",
    )
    st.tdr.add_model_labels(
        model = tail_tissue_pc,
        key_added = mk,
        labels = np.asarray(tail_tissue_adata[np.asarray(tail_tissue_pc.point_data["obs_index"])].obs[mk]),
        colormap = "afmhot_r",
        where = "point_data",
        inplace = True,
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tail_tissue_pc, vector_arrows]),
        key=[mk, mk],
        model_style=["points", "surface"],
        model_size=[16, 5],
        opacity=[0.2, 1],
        colormap=["afmhot_r", "afmhot_r"],
        show_legend=True,
        jupyter="static",
        background="white",
        cpo=cpo,
        window_size=(2560, 2048),
        text=mk,
        filename=os.path.join(out_image_path, f"CNS_tail_morphometrics_{mk}_arrows.pdf")
    )

In [58]:
morphometrics_keys = ["morpho_acceleration", "morpho_curvature", "morpho_curl", "morpho_torsion", "morpho_divergence"]
st.tdr.morphopath(
    adata=tail_tissue_adata,
    # layer="log1p_X",
    vf_key="VecFld_morpho",
    key_added="fate_morpho",
    t_end=10000,
    interpolation_num=50,
    cores=20
)
tail_trajectory_model, _ = st.tdr.construct_trajectory(
    adata=tail_tissue_adata,
    fate_key="fate_morpho",
    n_sampling=1000,
    sampling_method="trn",
    key_added="obs_index",
    label=np.asarray(tail_tissue_adata.obs.index), # stage1_tissue_adata.uns["VecFld_morpho"]["V"][:, 2].flatten(),
)
for mk in morphometrics_keys:
    st.tdr.add_model_labels(
        model=tail_trajectory_model,
        key_added=mk,
        labels=np.asarray(tail_tissue_adata[np.asarray(tail_trajectory_model.point_data["obs_index"])].obs[mk]),
        colormap="Spectral",
        where="point_data",
        inplace=True,
    )
    st.pl.three_d_plot(
        model=st.tdr.collect_models([tail_tissue_pc, tail_trajectory_model]),
        key=[mk, mk],
        model_style=["points", "wireframe"],
        model_size=[16, 5],
        opacity=[0.2, 0.5],
        colormap=["afmhot_r", "afmhot_r"],
        show_legend=True,
        jupyter="static",
        background="white",
        cpo=cpo,
        window_size=(2560, 2048),
        text=mk,
        filename=os.path.join(out_image_path, f"CNS_tail_morphometrics_{mk}_trajectory.pdf")
    )

## Save the anndata objects and models

In [59]:
del tail_tissue_adata.uns["morpho_torsion"], tail_tissue_adata.uns["fate_morpho"], tail_tissue_adata.uns["glm_degs"], tail_tissue_adata.uns["log1p"]
tail_tissue_adata.write_h5ad(os.path.join(out_h5ad_path, f"droso_{sample_id}_CNS_tail_v4.h5ad"), compression="gzip")
st.tdr.save_model(model=tail_tissue_pc, filename=os.path.join(out_h5ad_path, "CNS_tail_morphometrics_pc_model.vtk"))
st.tdr.save_model(model=tail_trajectory_model, filename=os.path.join(out_h5ad_path, "CNS_tail_morphometrics_trajectory_model.vtk"))
tail_tissue_adata